In [17]:
from graph import *
from models import *
from metrics import *

import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pickle as pkl

from scipy.optimize import curve_fit, approx_fprime
from multiprocess import Pool
from copy import deepcopy

In [2]:
# load data from the spreadsheet which defines the structure of the workflow,
# as well as the parameters for data rates, efficiency, data reduction, and classifier performance
# ...these are taken from predictions for the Run-5 CMS
run5_system = dataframes_from_spreadsheet("cms_system_200.xlsx")
run5_smartpx_system = dataframes_from_spreadsheet("cms_system_200_smartpx.xlsx")

In [3]:
run5_system.detectors

,Category,Detector,Data (bytes),Sample Rate,Compression,Link Efficiency (J/bit),Op Efficiency (J/op),PU 200
0,Tracking,Inner Tracker,1440000,40000000,0,2.220000e-11,0,1.440
1,Tracking,Outer Tracker PS,720000,40000000,0,2.220000e-11,0,0.720
2,Tracking,Outer Tracker 2S,430000,40000000,0,2.220000e-11,0,0.430
3,Tracking,Track Finder TPG,10000,40000000,0,2.220000e-11,0,0.010
4,Timing,MIP Timing BTL,240000,40000000,0,2.220000e-11,0,0.240
5,Timing,MIP Timing ETL,440000,40000000,0,2.220000e-11,0,0.440
6,Calorimetry,ECAL Barrel,600000,40000000,0,2.220000e-11,0,0.600
7,Calorimetry,HCAL Barrel,240000,40000000,0,2.220000e-11,0,0.240
8,Calorimetry,HCAL HO,30000,40000000,0,2.220000e-11,0,0.030
9,Calorimetry,HCAL HF,60000,40000000,0,2.220000e-11,0,0.060


In [9]:
#import the data predicting wall time scaling by pileup
scaling = pd.read_excel("wall time scaling.xlsx", sheet_name="Data")
#fit a polynomial to this data for CPU and GPU runtimes
fit_poly = lambda x, k3, k2, k1: k3 * x ** 3 + k2 * x ** 2 + k1 * x
k, cv = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time"])
k_gpu, cv_gpu = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time GPU"])

In [10]:
#define a dictionary with functions defining the scaling of trigger runtimes with incoming data
funcs = {"Global": lambda x: fit_poly(x, *k), "Intermediate": lambda x: x / 2.0e6}
funcs_gpu = {"Global": lambda x: fit_poly(x, *k_gpu), "Intermediate": lambda x: x / 2.0e6}

In [11]:
"""
Vary the accept rate of the level 1 trigger and inspect its impact on performance and resources required
"""
def init_system(system: System, functions, reduction: float):
    t = system.triggers.copy()
    #intermediate reduction stage
    t.at[4, "Reduction Ratio"] = reduction
    g = construct_graph(system.detectors, t, system.globals, functions)

    return g

In [12]:
reduction_to_ratio(0.0)

1.0

In [13]:
ex_gpu = init_system(run5_system, funcs_gpu, 53.3)

/Users/wilkie/code/system_flow/classifier.py:200: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])
/Users/wilkie/code/system_flow/classifier.py:424: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probabl

In [14]:
ex_reduction = init_system(run5_smartpx_system, funcs_gpu, 53.3)

/Users/wilkie/code/system_flow/classifier.py:200: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])
/Users/wilkie/code/system_flow/classifier.py:424: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probabl

In [15]:
#current l1t accept / skill
ex = init_system(run5_system, funcs, 400)

/Users/wilkie/code/system_flow/classifier.py:200: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])
/Users/wilkie/code/system_flow/classifier.py:424: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probabl

In [18]:
def vary_parameters(graph, tracker_data, hgcal_data, reduction_ratio, l1t_skill, hlt_skill, l1t_eff, hlt_eff):
    graph = deepcopy(graph)
    #system parameters
    graph.nodes["Inner Tracker"]["sample data"] = tracker_data
    graph.nodes["HGCAL"]["sample data"] = hgcal_data
    graph.nodes["Intermediate"]["reduction ratio"] = reduction_ratio

    #algorithm parameters
    graph.nodes["Intermediate"]["classifier"].skill_boost = l1t_skill
    graph.nodes["Global"]["classifier"].skill_boost = hlt_skill

    #tech parameters
    graph.nodes["Intermediate"]["op efficiency"] = l1t_eff
    graph.nodes["Global"]["op efficiency"] = hlt_eff

    #update the graph
    graph = update_throughput(graph)

    #calc productivity
    power = (graph.graph["op power"] + graph.graph["link power"]) / density_scale_model(2032)
    confusion = graph.graph["performance"]
    prod = (f1_score(confusion) * 7500) / power
    return prod
    

In [20]:
vp = lambda x: vary_parameters(ex_reduction, *x)

In [23]:
#run5 system model - tracker l1t upgrade w/ smart pixels

c_phase1 = [ex.nodes["Inner Tracker"]["message size"],
      ex.nodes["HGCAL"]["message size"],
      400, #reduction ratio
      0.0, #l1t skill boost
      0.0, #hlt skill boost
      ex.nodes["Intermediate"]["op efficiency"],
      ex.nodes["Global"]["op efficiency"],]


In [26]:
grad_phase1 = approx_fprime(c_phase1,
                vp,
                [1e2,
                 1e2,
                 10, 
                 0.05,
                 0.05,
                 1e-4,
                 1e-1],
              )

In [27]:
grad_phase1

array([-1.37673639e-10, -1.37673639e-10,  7.18830570e-07,  5.82528935e-04,
        0.00000000e+00, -3.54495550e-03, -2.92504738e-05])

In [ ]:
#run5 system model - tracker l1t upgrade w/ smart pixels

c_l1tracks = [ex.nodes["Inner Tracker"]["message size"],
      ex.nodes["HGCAL"]["message size"],
      400, #reduction ratio
      0.0, #l1t skill boost
      0.0, #hlt skill boost
      ex.nodes["Intermediate"]["op efficiency"],
      ex.nodes["Global"]["op efficiency"],]


In [28]:
#final system model - tracker l1t upgrade w/ smart pixels

c0a = [ex_reduction.nodes["Inner Tracker"]["message size"],
      ex_reduction.nodes["HGCAL"]["message size"],
      53.3, #reduction ratio
      0.4, #l1t skill boost
      0.0, #hlt skill boost
      ex_reduction.nodes["Intermediate"]["op efficiency"],
      ex_reduction.nodes["Global"]["op efficiency"],]


In [32]:
c_sphase2 = approx_fprime(c0a,
                vp,
                [1e2,
                 1e2,
                 15, 
                 0.05,
                 0.05,
                 1e-4,
                 1],
              )

In [42]:
c_sphase2 * 1e3

array([-9.15331776e-08, -9.15331776e-08,  5.21990212e-03,  1.32984327e-02,
        7.64453837e-03, -3.35877488e-01, -1.71109209e-02])

In [43]:
c_sphase2[4]

np.float64(7.64453836586904e-06)

In [ ]:
#HLT Skill / HLT energy
c_sphase2[4] / c_sphase2[6]

np.float64(-0.4467637015111934)

In [48]:
#L1T Skill / HLT Skill
c_sphase2[3] / c_sphase2[4]

np.float64(1.73959918130118)

In [ ]:
p_phase2 = np.float64(0.0002920143304319754)

In [50]:
#L1T Skill / HLT Skill
p_phase2 / c_sphase2[5]

np.float64(-0.8694072706745436)

In [54]:
#L1T Power gradient
c_sphase2[-2] * 1e6

np.float64(-335.8774883552692)

In [53]:
#HLT Power gradient
c_sphase2[-1]

np.float64(-1.711092091862237e-05)

In [35]:
relative_change = p_phase2 / c_sphase2

In [38]:
relative_change

array([-3.19025667e+06, -3.19025667e+06,  5.59424916e+01,  2.19585524e+01,
        3.81990797e+01, -8.69407271e-01, -1.70659622e+01])

In [ ]:
#HLT skill
relative_change[4]

np.float64(38.199079716277794)

In [ ]:
#HLT power
relative_change[-1]

np.float64(-17.065962248365413)